# AOC 2022

Welcome to the Advent of Code 2022 !

## Basic configuration

In [ ]:
!pip install aocd

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

from aocd import submit
from aocd.models import Puzzle

## Day 8
https://adventofcode.com/2022/day/8
### Part 1

In [ ]:
puzzle = Puzzle(2022, 8)
lines = puzzle.input_data.split('\n')

In [ ]:
forest = np.array([list(map(int, line)) for line in lines])

In [ ]:
import numpy as np

def visibility(line):
    top = -1
    mask = []
    for item in line:
        mask.append(top)
        top = max(top, item)
    return line > np.array(mask)

In [ ]:
left = np.array([visibility(line) for line in forest])
right = np.array([visibility(line[::-1])[::-1] for line in forest])
top = np.array([visibility(line) for line in forest.T]).T
bottom = np.array([visibility(line[::-1])[::-1] for line in forest.T]).T

answer_a = np.sum(left | right | top | bottom)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def los(line):
    if len(line) < 2:
        return 0
    
    top = line[0]
    
    for cnt, item in enumerate(line[1:]):
        if item >= top:
            break
    return cnt + 1

def viewing_score(x, y):
    # left, right, top, bottom
    return los(forest[y,x::-1]) * los(forest[y,x:]) * los(forest[y::-1,x]) * los(forest[y:,x])

In [ ]:
H, W = forest.shape
answer_b = max(viewing_score(x, y) for x in range(W) for y in range(H))

In [ ]:
puzzle.answer_b = answer_b

### Extra

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.imshow(forest, cmap='YlGn')
ax1.set_title("Forest")

ax2.imshow(left | right | top | bottom)
ax2.set_title("Visible trees")

## Day 7
https://adventofcode.com/2022/day/7
### Part 1

In [ ]:
puzzle = Puzzle(2022, 7)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import deque, defaultdict
import re
import os

class File:
    
    def __init__(self, name_, size_):
        self._name = name_
        self._size = size_
        
    def size(self):
        return int(self._size)

class Directory:
    
    def __init__(self):
        self.dirs = []
        self.files = []
        
    def size(self):
        return sum(file_.size() for file_ in self.files) + sum(dir_.size() for dir_ in self.dirs)
    

In [ ]:
def current_path():
    return os.path.join(*list(path))

path = deque()
inodes = defaultdict(Directory)

for line in lines:
    if line[0] == '$':
        cmd = line[2:]
        if cmd[:2] == 'cd':
            loc = cmd[3:]
            if loc == '..':
                path.pop()
            else:
                path.append(loc)
    else:
        cur_path = current_path()

        if line[:3] == 'dir':
            dir_path = os.path.join(cur_path, line[4:])
            inodes[cur_path].dirs.append(inodes[dir_path])
        else:
            inodes[cur_path].files.append(File(*line.split()[::-1]))

In [ ]:
answer_a = sum(dir_.size() for dir_ in inodes.values() if dir_.size() <= 100000)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
total_space = 70000000
target_free_space = 30000000

min_space_to_free = target_free_space - (total_space - inodes['/'].size())

In [ ]:
answer_b = min(dir_.size() for dir_ in inodes.values() if dir_.size() >= min_space_to_free)

In [ ]:
puzzle.answer_b = answer_b

### Part 2

## Day 6
https://adventofcode.com/2022/day/6
### Part 1

In [ ]:
puzzle = Puzzle(2022, 6)
lines = puzzle.input_data.split('\n')

In [ ]:
line = lines[0]
for pos in range(4,len(line)):
    if len(set(line[pos-4:pos])) == 4:
        break

In [ ]:
puzzle.answer_a = pos

### Part 2

In [ ]:
line = lines[0]
for pos in range(14,len(line)):
    if len(set(line[pos-14:pos])) == 14:
        break

In [ ]:
puzzle.answer_b = pos

## Day 5
https://adventofcode.com/2022/day/5
### Part 1

In [ ]:
puzzle = Puzzle(2022, 5)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import deque, defaultdict
import re

def init_crates(lines):
    crates = defaultdict(deque)

    for offset, line in enumerate(lines):
        if line[1] == '1':
            break
        for idx, elem in enumerate(line[1::4]):
            if elem != ' ':
                crates[idx].appendleft(elem)
    
    instructions = []

    for line in lines[offset+2:]:
        m = re.match(r"move (\d+) from (\d+) to (\d+)", line)
        instructions.append(tuple(map(int, m.groups())))
    
    return crates, instructions

In [ ]:
crates, instructions = init_crates(lines)

for nb, src, dst in instructions:
    crates[dst-1].extend([crates[src-1].pop() for i in range(nb)])

In [ ]:
answer_a = ''.join([crates[idx][-1] for idx in range(9)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
crates, instructions = init_crates(lines)

for nb, src, dst in instructions:
    crates[dst-1].extend([crates[src-1].pop() for i in range(nb)][::-1])

In [ ]:
answer_b = ''.join([crates[idx][-1] for idx in range(9)])

In [ ]:
puzzle.answer_b = answer_b

## Day 4
https://adventofcode.com/2022/day/4
### Part 1

In [ ]:
puzzle = Puzzle(2022, 4)
lines = puzzle.input_data.split('\n')

In [ ]:
ranges = []
for l in lines:
    pair = []
    for r in l.split(','):
        a, b = map(int, r.split('-'))
        pair.append(set(range(a,b+1)))
    ranges.append(pair)

In [ ]:
answer_a = len([r1 for r1, r2 in ranges if (r1 <= r2) or (r2 <= r1)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = len([r1 for r1, r2 in ranges if r1 & r2])

In [ ]:
puzzle.answer_b = answer_b

## Day 3
https://adventofcode.com/2022/day/3
### Part 1

In [ ]:
puzzle = Puzzle(2022, 3)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import Counter

sacks = [(Counter(l[:len(l)//2]), Counter(l[len(l)//2:])) for l in lines]

In [ ]:
def prio(c):
    if c.islower():
        return ord(c) - ord('a') + 1
    return ord(c) - ord('A') + 27

answer_a = sum(prio(list(s[0].keys() & s[1].keys())[0]) for s in sacks)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
sacks = [Counter(l) for l in lines]

answer_b = sum([prio(list(sacks[i].keys() & sacks[i+1].keys() & sacks[i+2].keys())[0]) for i in range(0, len(sacks), 3)])

In [ ]:
puzzle.answer_b = answer_b

## Day 2
https://adventofcode.com/2022/day/2
### Part 1

In [ ]:
puzzle = Puzzle(2022, 2)
lines = puzzle.input_data.split('\n')

In [ ]:
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    if a == b:
        score += 3
    if a == (b+1)%3:
        score += 0
    if a == (b+2)%3:
        score += 6
    score += b+1

In [ ]:
puzzle.answer_a = score

### Part 2

In [ ]:
## Short version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    score += 3*b + 1 + (a + (b+2) % 3) % 3

In [ ]:
## Long version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    
    score += 3*(b)

    score +=1
    if b == 0:
        score += (a+2)%3
    elif b==1:
        score += a
    elif b==2:
        score += (a+1)%3

In [ ]:
puzzle.answer_b = score

## Day 1
https://adventofcode.com/2022/day/1
### Part 1

In [ ]:
puzzle = Puzzle(2022, 1)

In [ ]:
lines = puzzle.input_data.split('\n\n')

In [ ]:
answer_a = max([sum(int(e) for e in l.split('\n')) for l in lines])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = sum(sorted([sum(int(e) for e in l.split('\n')) for l in lines], reverse=True)[:3])

In [ ]:
puzzle.answer_b = answer_b